In [1]:
import os

In [2]:
if "COLAB_GPU" in os.environ:
 #   !pip install PyMuPDF # for reading PDFs using python
  #  !pip install tqdm # for progress bars
   # !pip install accelerate # for quantization model loading
    #!pip install bitsandbytes # for quantization models (less storage/memory usage)
    !pip install flash-attn --no-build-isolation # for faster attention computation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 91.8 MB/s eta 0:00:00:00:010:01
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for flash-attn
  Running setup.py clean for flash-attn
Failed to build flash-attn
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (flash-attn)


In [3]:
!pip install flash-attn --no-build-isolation

  Using cached flash_attn-2.8.3.tar.gz (8.4 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for flash-attn
  Running setup.py clean for flash-attn
Failed to build flash-attn
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (flash-attn)


In [4]:
pip install PyMuPDF tqdm accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 95.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 16.3 MB/s eta 0:00:00:00:0100:01


In [5]:
#!pip uninstall -y torch torchvision torchaudio transformers sentence-transformers
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
#!pip install -U transformers sentence-transformers

In [6]:
import os
import requests

pdf_path = "Human-Nutrition.pdf"

if not os.path.exists(pdf_path):
    print("Downloading PDF...")

    url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

    filename = pdf_path
    response = requests.get(url)

    if response.status_code == 200:
        with open(filename, "wb") as f:
            f.write(response.content)
        print(f"Downloaded {filename}")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")
else:
    print(f"{pdf_path} already exists.")

Downloaded Human-Nutrition.pdf


In [7]:
import fitz # PyMuPDF

from tqdm.auto import tqdm

def text_formatter(text: str) -> str:
    # minor formatting to clean up text
    cleaned_text = text.replace("\n", " ").strip()
    return cleaned_text

def open_and_read_pdf(pdf_path: str) -> str:
    # Open the PDF file
    document = fitz.open(pdf_path)
    pages_and_texts = []

    # Iterate through each page and extract text
    for page_number, page in tqdm(enumerate(document)):
        #print(page_number, page)
        text = page.get_text()
        text = text_formatter(text)
        pages_and_texts.append({
        "page_number": page_number - 41,
        "page_char_count": len(text),
        "page_word_count": len(text.split(" ")),
        "Page_sentence_count_raw": len(text.split(". ")),
        "page_token_count": len(text)/4, # rough estimate: 1 token ~= 4 characters
        "text": text
        })

    return pages_and_texts


In [8]:
pages_and_texts = open_and_read_pdf(pdf_path = pdf_path)
pages_and_texts[:2]  # Display the first two pages' data

0it [00:00, ?it/s]

[{'page_number': -41,
  'page_char_count': 29,
  'page_word_count': 4,
  'Page_sentence_count_raw': 1,
  'page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': -40,
  'page_char_count': 0,
  'page_word_count': 1,
  'Page_sentence_count_raw': 1,
  'page_token_count': 0.0,
  'text': ''}]

In [9]:
import random

random_pages = random.sample(pages_and_texts, k=3)
random_pages

[{'page_number': 107,
  'page_char_count': 2280,
  'page_word_count': 379,
  'Page_sentence_count_raw': 16,
  'page_token_count': 570.0,
  'text': 'in the endocrine system are the pituitary, thyroid, parathyroid,  adrenals, thymus, pineal, pancreas, ovaries, and testes. The glands  secrete hormones, which are biological molecules that regulate  cellular processes in other target tissues, so they require  transportation by the circulatory system. Adequate nutrition is  critical for the functioning of all the glands in the endocrine system.  A protein deficiency impairs gonadal-hormone release, preventing  reproduction. Athletic teenage girls with very little body fat often  do not menstruate. Children who are malnourished usually do not  produce enough growth hormone and fail to reach normal height  for their age group. Probably the most popularized connection  between nutrition and the functions of the endocrine system is  that unhealthy dietary patterns are linked to obesity and the  

In [10]:
%pip install pandas

import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,Page_sentence_count_raw,page_token_count,text
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition
1,-40,0,1,1,0.00,
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,-38,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...
4,-37,797,145,2,199.25,Contents Preface University of Hawai‘i at Mā...


In [11]:
df.describe().round(2)

,page_number,page_char_count,page_word_count,Page_sentence_count_raw,page_token_count
count,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1148.00,198.30,9.97,287.00
std,348.86,560.38,95.76,6.19,140.10
min,-41.00,0.00,1.00,1.00,0.00
25%,260.75,762.00,134.00,4.00,190.50
50%,562.50,1231.50,214.50,10.00,307.88
75%,864.25,1603.50,271.00,14.00,400.88
max,1166.00,2308.00,429.00,32.00,577.00


Potentially each page can be embedded with embedding models of input capacity ~ 300

### Chunking

One sentence can be one chunk or one page can also be one chunk

Top 1 or top 3 chunks which are closest to the prompt are passed to the LLM.

### Testing 5 chunking strategies

#### Fixed size chunking

In [13]:
def chunk_text(text: str, chunk_size: int = 500)-> list:

    chunks = []
    current_chunk = ''
    words = text.split()

    for word in words:
        # check if adding the next word exceeds chunk size
        if len(current_chunk) + len(word) + 1 <= chunk_size:
            current_chunk += (word + ' ')
        else:
            # store cuurrent chunk and start a new one
            chunks.append(current_chunk.strip())
            current_chunk = word + ' '

    # add any remaining text as a chunk
    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks


In [12]:
def chunk_pdf_pages(pages_and_texts: list, chunk_size: int = 500) -> list[dict]:

    all_chunks = []
    for page in pages_and_texts:
        page_number = page['page_number']
        page_text = page['text']
        chunks = chunk_text(page_text, chunk_size=chunk_size)

        for i, chunk in enumerate(chunks):
            all_chunks.append({
                "page_number": page_number,
                "chunk_index": i,
                "chunk_text": chunk,
                "chunk_word_count": len(chunk.split()),
                "chunk_char_count": len(chunk),
                "chunk_token_count": len(chunk)/4  # rough estimate
            })

    return all_chunks
    

In [19]:
chunked_pages = chunk_pdf_pages(pages_and_texts, chunk_size=500)
print(f"Total chunks created: {len(chunked_pages)}")
print(f"First chunk: (page {chunked_pages[0]['page_number']}) {chunked_pages[0]['chunk_text'][:200]}")

Total chunks created: 3321
First chunk: (page -41) Human Nutrition: 2020 Edition


In [21]:
chunked_pages[500]

{'page_number': 165,
 'chunk_index': 3,
 'chunk_text': 'activity level, and kidney function. No maximum for water intake has been set. Regulation of Water Balance | 165',
 'chunk_word_count': 19,
 'chunk_char_count': 112,
 'chunk_token_count': 28.0}

In [30]:
import random
[random.randrange(100)]

[24]

#### Semantic chunking

In [31]:
!pip -q install --upgrade "sentence-transformers==3.0.1" "transformers<5,>=4.41" scikit-learn nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 126.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 85.9 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk
nltk.download('punkt', quiet=True)

semantic_model = SentenceTransformer('all-MiniLM-L6-v2')

def semantic_chunk_text(text: str, similairty_threshold: float = 0.8, max_tokens: int = 500) -> list:
    """"
    Splits text into semantic chunks based on similarity threshold and max token limit.
    """
    sentences = nltk.sent_tokenize(text)
    if not sentences:
        return []
    
    embeddings = semantic_model.encode(sentences)

    chunks = []
    current_chunk = [sentences[0]]
    current_embedding = embeddings[0]

    for i in range(1, len(sentences)):
        sim = cosine_similarity([current_embedding], [embeddings[i]])[0][0]
        chunked_token_count = len(" ".join(current_chunk)) // 4  # rough estimate

        if sim>= similairty_threshold and chunked_token_count< max_tokens:
            current_chunk.append(sentences[i])
            current_embedding = np.mean([current_embedding, embeddings[i]], axis=0)


    return embeddings

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [34]:
semantic_chunk_text(chunked_pages[500]['chunk_text'])

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
